In [2]:
import requests
def download_file_with_custom_headers(url, local_filename):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()  # This will throw an exception for non-200 responses
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print("File downloaded successfully.")


# Example usage
file_url = 'https://fesb-my.sharepoint.com/:u:/g/personal/toperkov_fesb_hr/ERP3tpm9FRRIkk82lCHbQpIBGu-9efbxohQv6dZ6g2B2AQ?download=1'
output_dir = './imageFESB.001'
download_file_with_custom_headers(file_url, output_dir)

Bitstream image verified successfully.


True

In [5]:
import hashlib

def verify_image_hash(image_path, given_hash):
    """
    Verifies the integrity of an image file by comparing its SHA-1 hash against a given hash.

    Parameters:
    - image_path (str): The file path to the image whose integrity is being verified.
    - given_hash (str): The expected SHA-1 hash value for the image file.

    Returns:
    - bool: True if the image's SHA-1 hash matches the given hash, False otherwise.
    """
    try:
        # Compute the SHA-1 hash of the image file
        with open(image_path, 'rb') as f:
            computed_hash = hashlib.sha1(f.read()).hexdigest()

        # Compare the computed hash with the given hash
        if computed_hash == given_hash:
            print('Bitstream image verified successfully.')
            return True
        else:
            print('Error: Bitstream image verification failed.')
            return False
    except IOError as e:
        print(f"Error opening file: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

# Example usage
image_path = 'imageFESB.001'
given_hash = "201cdee056cfc8c0996328e3c2115b513a141f5c"
verify_image_hash(image_path, given_hash)

Bitstream image verified successfully.


True

In [8]:
from pyunpack import Archive
import os

def create_directory(directory_path):
    try:
        os.makedirs(directory_path, exist_ok=True)
        print(f"Directory '{directory_path}' created or already exists.")
    except Exception as e:
        print(f"Failed to create directory '{directory_path}': {e}")

# Example usage
directory_path = 'john-1.9.0-jumbo-1-win64'
create_directory(directory_path)

Archive('john-1.9.0-jumbo-1-win64.zip').extractall("john-1.9.0-jumbo-1-win64")

Directory 'john-1.9.0-jumbo-1-win64' created or already exists.


In [1]:
import socket
import subprocess

# Set the path to the USB image file
image_path = 'imageFESB.001'

# Call bitlocker2john to extract the recovery key
bitlocker2john_cmd = r'C:\Users\A507\RF\lcupic\john-1.9.0-jumbo-1-win64\john-1.9.0-jumbo-1-win64\run\bitlocker2john.exe -i imageFESB.001'
process = subprocess.Popen(bitlocker2john_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
output, error = process.communicate()

# Print the extracted recovery key
keys = output.decode().strip().split('\n')
recovery_key = [s for s in keys if "$bitlocker$1$" in s]
print(f'BitLocker recovery key: {recovery_key[0]}')

BitLocker recovery key: $bitlocker$1$16$f64d2073dea16d4bf96c9e4622f4b08e$1048576$12$402ff33f651ad70103000000$60$265c4b1591ba8b3c7e8f6401657479ba7cc05caba44cdfe1d25a10c0099617698226aa2f9ca826fb074ce7ad9738be9bbc95a6b516527d2632af7d6f


In [19]:
output_file = "hash.txt"
hashcat_cmd = f'hashcat -m 22100 -a 3 {recovery_key[0]} --potfile-disable --remove --outfile {output_file} "218546?d?d"'
process = subprocess.call(hashcat_cmd, shell=True)

# Execute hashcat command using subprocess
try:
    output = subprocess.check_output(hashcat_cmd, shell=True)
    print(output.decode("utf-8"))
except subprocess.CalledProcessError as e:
    print(f"Error executing hashcat command: {e}")